<a href="https://colab.research.google.com/github/GDAmitha/plasmidInteractions/blob/main/GeneBank_Parsing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Required Packages:

In [ ]:
!pip install biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 11.8 MB/s eta 0:00:00


In [ ]:
from Bio import SeqIO
from Bio.SeqUtils import GC
import collections

import requests
import gzip
import os
from Bio import SeqIO

from google.colab import drive


Function that takes a given genebank file that has been read by SeqIO and parses through it, collecting different respresentations of the sequence.

In [ ]:
def analyze_gene_sequence(genbank_file):
    # Parse the GenBank file
    #record = SeqIO.read(genbank_file, "genbank")
    full_sequence = str(record.seq)

    # Extract features
    feature_list = [{"type": feature.type, "location": str(feature.location)} for feature in record.features]

    # Gene structure (simplified for this example)
    gene_structure = {"coding_sequences": [], "introns": [], "exons": []}
    for feature in record.features:
        if feature.type == "CDS":
            gene_structure["coding_sequences"].append(str(feature.location))
        # Introns and exons can be added similarly

    # K-mer distribution (example with k=3)
    kmer_size = 3
    kmer_distribution = collections.Counter([full_sequence[i:i+kmer_size] for i in range(len(full_sequence) - kmer_size + 1)])

    # Physicochemical properties
    physicochemical_properties = {"GC_content": GC(record.seq)}

    # Compressed sequence and sketching are more complex and may require additional libraries or custom algorithms

    # Compile all information into a dictionary
    gene_data = {
        "full_sequence": full_sequence,
        "feature_list": feature_list,
        "gene_structure": gene_structure,
        "kmer_distribution": kmer_distribution,
        "physicochemical_properties": physicochemical_properties
        # Add other representations as needed
    }

    return gene_data


Function to download and decompress the file directly from the genebank website.


In [ ]:

def download_and_decompress(url, local_filename):
    # Download the file
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)

    # Decompress the file
    with gzip.open(local_filename, 'rb') as f_in:
        with open(local_filename[:-3], 'wb') as f_out:
            f_out.write(f_in.read())

    os.remove(local_filename)  # Remove the compressed file


Main function to parse through 5 sequences.

In [ ]:
# Base URL for GenBank sequence files
base_url = "https://ftp.ncbi.nih.gov/genbank/"

In [ ]:

# Files to download and parse
files = ["gbbct1001.seq.gz", "gbbct1002.seq.gz", "gbbct1003.seq.gz", "gbbct1004.seq.gz", "gbbct1005.seq.gz"]

files_to_genedata = {}

for file in files:
    url = base_url + file
    local_filename = file
    print(f"Downloading and decompressing {file}...")
    download_and_decompress(url, local_filename)

    # Parse the decompressed file
    print(f"Parsing {local_filename[:-3]}...")
    with open(local_filename[:-3], "r") as handle:
        for record in SeqIO.parse(handle, "genbank"):
            gene_data = analyze_gene_sequence(record)

            gene_data = analyze_gene_sequence(record)
            files_to_genedata[record.name] = gene_data
            pass

    os.remove(local_filename[:-3])  # Remove the decompressed file



Parsing gbbct1001.seq...


/usr/local/lib/python3.10/dist-packages/Bio/SeqUtils/__init__.py:144: BiopythonDeprecationWarning: GC is deprecated; please use gc_fraction instead.
  warnings.warn(


Parsing gbbct1002.seq...
Parsing gbbct1003.seq...
Parsing gbbct1004.seq...
Parsing gbbct1005.seq...


In [ ]:
files2 = ["gbbct1002.seq.gz"]

files_to_genedata2 = {}

for file in files2:
    url = base_url + file
    local_filename = file
    print(f"Downloading and decompressing {file}...")
    download_and_decompress(url, local_filename)

    # Parse the decompressed file
    print(f"Parsing {local_filename[:-3]}...")
    with open(local_filename[:-3], "r") as handle:
        for record in SeqIO.parse(handle, "genbank"):
            gene_data = analyze_gene_sequence(record)

            gene_data = analyze_gene_sequence(record)
            files_to_genedata2[record.name] = gene_data
            pass

    os.remove(local_filename[:-3])  # Remove the decompressed file

Parsing gbbct1002.seq...


/usr/local/lib/python3.10/dist-packages/Bio/SeqUtils/__init__.py:144: BiopythonDeprecationWarning: GC is deprecated; please use gc_fraction instead.
  warnings.warn(


Function to aggregate a specific set of features we want, like CDS and gene sequences.

In [ ]:
def aggregate_gene_features(gene_data_dict):
    # Initialize an aggregate dictionary for feature lists
    aggregate_features = {"source": [], "gene": [], "rRNA": [], "CDS": []}

    # Iterate through each gene entry in the large dictionary
    for gene_id, gene_info in gene_data_dict.items():
        # Extract the feature list for the current gene
        feature_list = gene_info.get("feature_list", [])

        # Aggregate features
        for feature in feature_list:
            feature_type = feature["type"]
            if feature_type in aggregate_features:
                aggregate_features[feature_type].append((gene_id, feature["location"]))

    return aggregate_features

In [ ]:

aggregated_features = aggregate_gene_features(files_to_genedata2)


In [ ]:
aggregated_features

{'source': [('LS483348', '[0:1966581](+)'),
  ('LS483349', '[0:2019343](+)'),
  ('LS483350', '[0:2827594](+)'),
  ('LS483351', '[0:1898595](+)'),
  ('LS483352', '[0:1857727](+)'),
  ('LS483353', '[0:1766320](+)'),
  ('LS483354', '[0:2071453](+)'),
  ('LS483355', '[0:1752074](+)'),
  ('LS483356', '[0:1931144](+)'),
  ('LS483357', '[0:1891095](+)'),
  ('LS483358', '[0:2335264](+)')],
 'gene': [('LS483348', '[0:1356](+)'),
  ('LS483348', '[1514:2651](+)'),
  ('LS483348', '[2781:3006](+)'),
  ('LS483348', '[3039:3498](+)'),
  ('LS483348', '[3582:3774](+)'),
  ('LS483348', '[3878:4994](+)'),
  ('LS483348', '[5069:5639](+)'),
  ('LS483348', '[5631:9141](+)'),
  ('LS483348', '[9266:9794](+)'),
  ('LS483348', '[9930:10203](+)'),
  ('LS483348', '[10189:10561](+)'),
  ('LS483348', '[10697:11984](+)'),
  ('LS483348', '[11994:13260](+)'),
  ('LS483348', '[13264:13807](+)'),
  ('LS483348', '[13827:15801](+)'),
  ('LS483348', '[16328:17865](+)'),
  ('LS483348', '[17922:17997](+)'),
  ('LS483348', '[

Now this aggregated data has a list of the locations of the various genes and cds sequences of 11 different genomes.

Function to map this locations to sequence subsets.

In [ ]:
def extract_subsequences(aggregated_features, gene_data_dict, feature_type='gene', start=0, limit=20):
    extracted_subsequences = []

    for gene_id, location in aggregated_features.get(feature_type, [])[start:start+limit]:
        # Find the full sequence in gene_data_dict
        full_sequence = gene_data_dict.get(gene_id, {}).get("full_sequence", "")

        # Extract the location range
        # The format is assumed to be "[<start:>end](+/-)"
        loc_parts = location.strip("[]()").split(":")
        start_part, end_part = loc_parts[0], loc_parts[1]

        # Parsing start and end indices, handling cases like '<start' and '>end'
        start = int(''.join(filter(str.isdigit, start_part)))
        end = int(''.join(filter(str.isdigit, end_part)))

        # Extract the subsequence
        subsequence = full_sequence[start:end]

        # Append the tuple (source, subsequence)
        extracted_subsequences.append((gene_id, subsequence))

    return extracted_subsequences

# Example usage
# Assuming 'aggregated_features' and 'gene_data_dict' are defined
subsequences = extract_subsequences(aggregated_features, files_to_genedata2)
print(subsequences)


[('LS483348', 'ATGACTGAAAATGAACAAATTTTTTGGAATAGGGTCCTTGAACTGGCAAAAAGTCAACTAAAACAAGCCACTTATGAATTTTTTGTTTTAGATGCTCGATTAATTCAAATTGAGCAAAATACGGCGACGATTTACCTGGATCCTATGAAAGAACTCTTTTGGGATAAAAATTTAAAACCAATCATTTTAACGGCTGGTTTTGAGGTTTATAATACTGAAATTGTCGTGAACTATGTCTTTGAAGAAGATTTAGCTAAACAAGCAGTAGAAGAACCAACTTCCCAAGTTCTCCAAGCCCCACAAAAGAATCACCTGCCACAGGTTGATTCAGATTTAAATACAAAGTATACTTTTGACAACTTTGTCCAAGGTGATGAAAACCGTTGGGCCTTTTCTGCGTCTTATGCCGTTGCGGATGCTCCAGGAACTACTTACAACCCTTTATTTATCTGGGGTGGACCTGGGCTCGGAAAAACTCACTTGCTAAATGCCATTGGTAATGCGGTATTGCAAAATAATCCTAAAGCGCGCGTGAAGTACATCACAGCTGAAAATTTCATCAATGAATTTGTTATCCATATTCGACTGGATACTATGGAAGAATTGAAAGAAAAATTCCGTAATCTTGATGTTTTGCTGATTGATGACATTCAATCGCTCGCCAAAAAAACATTATCTGGTACGCAAGAAGAGTTTTTCAATACTTTCAACGCTCTTTACGATAACAACAAACAAATCGTACTAACCAGTGACCGCACACCAGATCACCTCGATAATCTGGAACAACGTTTGGTCACGCGCTTCAAATGGGGCTTGACAATCAATATCACGCCGCCTGATTTTGAAACACGTGTGGCAATTTTGACCAATAAAACGCAAGAATACGATTTTGTGTTCCCGCAGGATACCATTGAATATCTTGCTGGACAATTTGATTCTAACGTCCGTGACCTAGAAGGTGCTTTAAAGGATATTAGCCTTGTCG

#Testing performance of workflow:

In [ ]:
import time

def timed_function(func, *args, **kwargs):
    start_time = time.time()
    result = func(*args, **kwargs)
    end_time = time.time()
    return result, end_time - start_time

def test_extract_subsequences_performance(aggregated_features, gene_data_dict):
    limits = range(20, 2001, 20)  # From 20 to 2000 in steps of 20
    times = []

    for limit in limits:
        _, duration = timed_function(extract_subsequences, aggregated_features, gene_data_dict, 'gene', 0, limit)
        print(f"Time taken for limit {limit}: {duration} seconds")
        times.append((limit, duration))

    return times

# Example usage
# Assuming 'aggregated_features' and 'gene_data_dict' are defined
performance_data = test_extract_subsequences_performance(aggregated_features, files_to_genedata2)


Time taken for limit 20: 0.00011539459228515625 seconds
Time taken for limit 40: 0.0001862049102783203 seconds
Time taken for limit 60: 0.0004439353942871094 seconds
Time taken for limit 80: 0.0005152225494384766 seconds
Time taken for limit 100: 0.0009882450103759766 seconds
Time taken for limit 120: 0.0005242824554443359 seconds
Time taken for limit 140: 0.0010738372802734375 seconds
Time taken for limit 160: 0.00067138671875 seconds
Time taken for limit 180: 0.0008146762847900391 seconds
Time taken for limit 200: 0.0008361339569091797 seconds
Time taken for limit 220: 0.0006399154663085938 seconds
Time taken for limit 240: 0.0006384849548339844 seconds
Time taken for limit 260: 0.0007216930389404297 seconds
Time taken for limit 280: 0.0007395744323730469 seconds
Time taken for limit 300: 0.0007576942443847656 seconds
Time taken for limit 320: 0.0008006095886230469 seconds
Time taken for limit 340: 0.0008711814880371094 seconds
Time taken for limit 360: 0.0009210109710693359 seconds


Extracting sequences from our dictionary data format is relatively efficient.

New files list for aggregation performance testing.

In [ ]:

files3 = ["gbbct1001.seq.gz", "gbbct1003.seq.gz", "gbbct1004.seq.gz", "gbbct1005.seq.gz"]

files_to_genedata3 = {}
times = {}

for file in files3:
    start_time = time.time()

    url = base_url + file
    local_filename = file
    print(f"Downloading and decompressing {file}...")
    download_and_decompress(url, local_filename)

    print(f"Parsing {local_filename[:-3]}...")
    with open(local_filename[:-3], "r") as handle:
        for record in SeqIO.parse(handle, "genbank"):
            gene_data = analyze_gene_sequence(record)
            files_to_genedata3[record.name] = gene_data

    os.remove(local_filename[:-3])  # Remove the decompressed file

    end_time = time.time()
    times[file] = end_time - start_time

# Print out the times for each file
for file, duration in times.items():
    print(f"Time taken for {file}: {duration} seconds")


Parsing gbbct1001.seq...


/usr/local/lib/python3.10/dist-packages/Bio/SeqUtils/__init__.py:144: BiopythonDeprecationWarning: GC is deprecated; please use gc_fraction instead.
  warnings.warn(


Parsing gbbct1003.seq...
Parsing gbbct1004.seq...
Parsing gbbct1005.seq...
Time taken for gbbct1001.seq.gz: 145.01077699661255 seconds
Time taken for gbbct1003.seq.gz: 144.48458623886108 seconds
Time taken for gbbct1004.seq.gz: 159.60398030281067 seconds
Time taken for gbbct1005.seq.gz: 161.49549746513367 seconds


In [ ]:
import random

def test_aggregate_gene_features_performance(original_gene_data_dict, test_sizes):
    times = []

    for size in test_sizes:
        # Create a subset of the original dictionary of the specified size
        subset_dict = dict(random.sample(original_gene_data_dict.items(), size))

        _, duration = timed_function(aggregate_gene_features, subset_dict)
        print(f"Time taken for input size {size}: {duration} seconds")
        times.append((size, duration))

    return times

# Example usage
# Assuming 'original_gene_data_dict' is your full gene data dictionary
test_sizes = [100, 200, 500, 1000, 1500, 2000]  # Varying sizes of input dictionary
performance_data = test_aggregate_gene_features_performance(files_to_genedata3, test_sizes)


Time taken for input size 100: 0.033856868743896484 seconds


<ipython-input-30-22a0975c45a6>:8: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  subset_dict = dict(random.sample(original_gene_data_dict.items(), size))


Time taken for input size 200: 0.07692575454711914 seconds
Time taken for input size 500: 0.14785218238830566 seconds
Time taken for input size 1000: 0.2236640453338623 seconds
Time taken for input size 1500: 0.6233797073364258 seconds
Time taken for input size 2000: 0.655585527420044 seconds


This shows how, once we have loaded our genome sequences in the dictionary database, extracting exclusive genes is also efficient.

#Compiling data into json files.

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


Function + main to store data as a json file.

In [ ]:
def write_gene_data_to_file(files_to_genedata, filename):
    with open(filename, 'w') as file:
        json.dump(files_to_genedata, file, indent=4)

In [ ]:
import json
write_gene_data_to_file(files_to_genedata, 'gene_data_output.json')

The output file is currently 1 GB rn!

##Compressing the data for accessibility

Downloading locally if necessary

In [ ]:
from google.colab import files

files.download('gene_data_output.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

saving as a compressed file

In [ ]:

# Path in your Google Drive where you want to save the file
# path_to_save = '/content/drive/My Drive/gene_data_output.json'

# # Assuming 'all_feature_lists' is your data
# with open(path_to_save, 'w') as file:
#     json.dump(all_feature_lists, file, indent=4)

import gzip
import shutil

with open('/content/drive/My Drive/gene_data_output.json', 'rb') as f_in:
    with gzip.open('/content/drive/My Drive/gene_data_output.json.gz', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
/content/gene_data_output.json